<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>
Logistic Regression using Python (stasmodel.formula):</p><br>
<p style="font-family: Arial; font-size:2.25em;color:green; font-style:bold"><br>
Kumar Rahul</p><br>

### We will be using HR data in this exercise. Refer the Exhibit 1 to understand the feature list. Use the HR data and answer the below questions.

1.	Load the dataset in Jupyter Notebook using pandas
2.	Build a correlation matrix between all the numeric features in the dataset. Report the features, which are correlated at a cut-off of 0.70. What actions will you take on the features, which are highly correlated?
3.	Build a new feature named LOB_Hike_Offered using LOB and percentage hike offered. Include this as a part of the data frame created in step 1. What assumption are you trying to test with such variables?
4.	Create a new data frame with the numeric features and categorical features as dummy variable coded features. Which features will you include for model building and why?
5.	Split the data into training set and test set. Use 80% of data for model training and 20% for model testing. 
6.	Build a model using Gender and Age as independent variable and Status as dependent variable.
    >* Are Gender and Age a significant feature in this model?
    * What inferences can be drawn from this model? 
7.	Build a model with statsmodel.api to predict the probability of Not Joining. How do you interpret the model outcome? Report the model performance on the test set.
8.	Build a model with statsmodel.formula.api to predict the probability of Not Joining and report the model performance on the test set. What difference do you observe in the model built here and the one built in step 7.
9.	Build a model using sklearn package to predict the probability of Not Joining. What difference do you observe in this model compared to model built in step 7 and 8.
10.	Fine-tune the cut-off value using cost of misclassification as a strategy. The cut-off should help classify maximum number of Not Joining cases correctly.
11.	Fine-tune the cut-off value using youdens index as a strategy. The cut-off should help balance the classification of Joined and Not Joined cases.
12.	Apply the cut-off values obtained in step 10 and step 11 on the test set. What inference can be deduced from it?
13. Build model using gradient descent to get an intuition about the inner working of optimization algorithms.
14. Build model using gradient descent with regularization to get an intution about the inner working of optimization algorithms.

**PS: Not all the questions are being answered as a part of the same notebook. You are encouraged to answer the questions if you find them missing.**

**Exhibit 1**


|Sl.No.|Name of Variable|Variable Description|
|:-------|----------------|:--------------------|
|1	|Candidate reference number|	Unique number to identify the candidate|
|2	|DOJ extended|Binary variable identifying whether candidate asked for date of joining extension (Yes/No)|
|3	|Duration to accept the offer|	Number of days taken by the candidate to accept the offer (continuous variable)|
|4	|Notice period|	Notice period to be served in the parting company before candidate can join this company (continuous variable)|
|5	|Offered band|	Band offered to the candidate based on experience and performance in interview rounds (categorical variable labelled C0/C1/C2/C3/C4/C5/C6)|
|6	|Percentage hike (CTC) expected|	Percentage hike expected by the candidate (continuous variable)|
|7	|Percentage hike offered (CTC)| Percentage hike offered by the company (continuous variable)|
|8	|Percent difference CTC|	Percentage difference between offered and expected CTC (continuous variable)|
|9	|Joining bonus|	Binary variable indicating if joining bonus was given or not (Yes/No)|
|10	|Gender|	Gender of the candidate (Male/Female)|
|11	|Candidate source|	Source from which resume of the candidate was obtained (categorical variables with categories  Employee referral/Agency/Direct)|
|12	|REX (in years)|	Relevant years of experience of the candidate for the position offered (continuous variable)|
|13	|LOB|	Line of business for which offer was rolled out (categorical variable)|
|14	|DOB|	Date of birth of the candidate|
|15	|Joining location|	Company location for which offer was rolled out for candidate to join (categorical variable)|
|16	|Candidate relocation status|	Binary variable indicating whether candidate has to relocate from one city to another city for joining (Yes/No)|
|17 |HR status|	Final joining status of candidate (Joined/Not-Joined)|



***

# Code starts here

To know the environment with the pyhton kernal



In [ ]:
import sys, os

sys.executable

Supress the warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

We are going to use below mentioned libraries for **data import, processing and visulization**. As we progress, we will use other specific libraries for model building and evaluation. 

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sn # visualization library based on matplotlib
import matplotlib.pylab as plt

#the output of plotting commands is displayed inline within Jupyter notebook
%matplotlib inline 


## Data Import and Manipulation

### 1. Importing a data set

_Give the correct path to the data_



modify the ast_note_interactivity kernel option to see the value of multiple statements at once.

In [ ]:
import os

os.getcwd()

#os.chdir()

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
raw_df = pd.read_csv( "../HR_case/data/IMB533_HR_Data_No_Missing_Value.csv", 
                        sep = ',', na_values = ['', ' '])

raw_df.columns = raw_df.columns.str.lower().str.replace(' ', '_')
raw_df.head()

In [ ]:
#?pd.read_csv

Dropping SLNo and Candidate.Ref as these will not be used for any analysis or model building.

In [ ]:
#?raw_df.drop()

In [ ]:
if set(['slno','candidate_ref']).issubset(raw_df.columns):
    raw_df.drop(['slno','candidate_ref'],axis=1, inplace=True)
    
raw_df.head()


### 2. Structure of the dataset



In [ ]:
raw_df.info()

In [ ]:
raw_df.status.value_counts()
#raw_df.describe(include='all').transpose()
raw_df.describe().transpose()

To get a help on the features of a object

In [ ]:
#?raw_df.status.value_counts()

### 2. Summarizing the dataset
Create a new data frame and store the raw data copy. This is being done to have a copy of the raw data intact for further manipulation if needed. The *dropna()* function is used for row wise deletion of missing value. The axis = 0 means row-wise, 1 means column wise.


In [ ]:
filter_df = raw_df.dropna(axis=0, how='any', thresh=None, 
                             subset=None, inplace=False)

list(filter_df.columns )

In [ ]:
#?raw_df.dropna

We will first start by printing the unique labels in categorical features

In [ ]:
numerical_features = ['duration_to_accept_offer','notice_period','pecent_hike_expected_in_ctc',
                      'percent_hike_offered_in_ctc','percent_difference_ctc','rex_in_yrs','age']

categorical_features = ['doj_extended','offered_band','joining_bonus','candidate_relocate_actual',
                        'gender','candidate_source','lob','location','status']

for f in categorical_features:
    print("\nThe unique labels in {} is {}\n".format(f, filter_df[f].unique()))
    print("The values in {} is \n{}\n".format(f,  filter_df[f].value_counts()))


Looking at the feature **line of business** it seems that *EAS, Healthcare and MMS* does not have enough observations and may be clubbed together

In [ ]:
filter_df['lob']=np.where(filter_df['lob'] =='EAS', 'Others', filter_df['lob'])
filter_df['lob']=np.where(filter_df['lob'] =='Healthcare', 'Others', filter_df['lob'])
filter_df['lob']=np.where(filter_df['lob'] =='MMS', 'Others', filter_df['lob'])
filter_df.lob.value_counts()

We will use **groupby** function of pandas to get deeper insights of the behaviour of people **Joining** or **Not Joining** the company. We will write a generic function to report the mean by any categorical variable.

In [ ]:
def group_by (categorical_features):
    return filter_df.groupby(categorical_features).mean()



In [ ]:
group_by("doj_extended")
group_by("status")
group_by("location")

### 3. Visualizing the Data

Plot can be done using the callable functions of 

>1. pandas library (http://pandas.pydata.org/pandas-docs/stable/visualization.html)
2. matplotlib library (https://matplotlib.org/) or
3. seaborn library (https://seaborn.pydata.org/) which is based on matplotlib and provides interface for drawing attractive statistical graphics.

#### 3a. Visualizing the Data using seaborn

Write a custom function to create bar plot to visualize the average of numeric features w.r.t each categorical feature. Say, average number of days to accept the offer w.r.t status as joined vs. not joined.

In [ ]:
def bar_plot(xlabel,ylabel):
    sn.barplot(x = xlabel, y = ylabel, data= filter_df)
    plt.xlabel(xlabel, size = 14)
    plt.ylabel(ylabel, size = 14)
    #plt.grid(True)
    x1,x2,y1,y2 = plt.axis()
    plt.show()

In [ ]:
numerical_features_set = ['duration_to_accept_offer','notice_period','age']
categorical_features_set = ['offered_band','gender','status']

for c in categorical_features_set:
    for n in numerical_features_set:
        bar_plot(c,n)

## Model:  Without dummy variable coding (Using Statsmodel.formula.api)

This is a code with R and Python being used in the same notebook

Refer for details: https://www.statsmodels.org/dev/example_formulas.html and https://www.statsmodels.org/dev/examples/notebooks/generated/glm_formula.html

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

To print the name of all the models in any library

In [ ]:
#dir(smf)

### 3. Create train and test dataset

#### Train and test data split using Python

The train and test split can also be done using the **sklearn module**

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split( filter_df, test_size = 0.2, random_state = 42)

We can pull the specific attribute needed to build the model is another data frame. This again is more of a hygine practice to not touch the **train** and **test** data set directly.

In [ ]:
lg_train_df = train_df.filter(['duration_to_accept_offer',
                               'notice_period',
                               'percent_difference_ctc',
                               'rex_in_yrs',
                               'age',
                               'doj_extended',
                               'offered_band',
#                               'pecent_hike_expected_in_ctc',
#                               'percent_hike_offered_in_ctc',
                               'joining_bonus',
#                               'candidate_relocate_actual',
                               'gender',
                               'candidate_source',
                               'lob',
                               'location',
                               'status'
                              ], axis = 1)

lg_test_df = test_df.filter(['duration_to_accept_offer',
                               'notice_period',
                               'percent_difference_ctc',
                               'rex_in_yrs',
                               'age',
                               'doj_extended',
                               'offered_band',
#                               'pecent_hike_expected_in_ctc',
#                               'percent_hike_offered_in_ctc',
                               'joining_bonus',
#                               'candidate_relocate_actual',
                               'gender',
                               'candidate_source',
                               'lob',
                               'location',
                               'status'
                              ], axis = 1)

lg_train_df.info()

Writing the formula with the required set of variables to be used in model building

In [ ]:
pass_formula = 'C(status) ~ \
            duration_to_accept_offer+ \
            notice_period + \
            percent_difference_ctc + \
            rex_in_yrs + \
            age + \
            C(doj_extended) + \
            C(offered_band) + \
            C(joining_bonus) + \
            C(gender) + \
            C(candidate_source) + \
            C(lob) + \
            C(location)'

In [ ]:
lg_reg_model = smf.glm(formula=pass_formula, data=lg_train_df, family=sm.families.Binomial()).fit()
lg_reg_model.summary()

## Find the significant variables


In [ ]:
def get_significant_vars (modelobject):
    var_p_vals_df = pd.DataFrame(modelobject.pvalues)
    var_p_vals_df['vars'] = var_p_vals_df.index
    var_p_vals_df.columns = ['pvals', 'vars']
    return list(var_p_vals_df[var_p_vals_df.pvals <= 0.05]['vars'])

In [ ]:
significant_vars = get_significant_vars(lg_reg_model)
significant_vars


## Model Evaluation


### 1. The prediction on train data.
Two ways to precit the outcome on the **train set**
> * Use **predict** function of the model object 
* Use **get_prediction** function of the model object

For the model with dummy variable coding explicetely done, we need to add the constant term to the test set. For the model with dummy variable coding carried out automatically, there is no need to add the constant term to the test set.

Here is the output with the model with no dummy variable coding

In [ ]:
predict_train_df = lg_reg_model.predict((lg_train_df))
predict_train_df.head()

predict_train_df = lg_reg_model.get_prediction(lg_train_df)
predict_train_df.predicted_mean[0:5]

### 2. The prediction on test data.

The prediction can be carried out by **defining functions** as well. Below is one such instance wherein a function is defined and is used for prediction

In [ ]:
def get_predictions ( test_class, model, test_data ):
    y_pred_df = pd.DataFrame( { 'actual': test_class,
                               'predicted_prob': model.get_prediction((test_data)).predicted_mean})
    return y_pred_df

In [ ]:
predict_test_df = pd.DataFrame(get_predictions(lg_test_df.status, lg_reg_model, lg_test_df))
predict_test_df.head()

In [ ]:
predict_test_df['predicted'] = predict_test_df.predicted_prob.map(lambda x: 'Joined' if x > 0.5 else 'Not Joined')
predict_test_df[0:10]


### 3. Confusion Matrix

We will built classification matrix using the **metrics** method from **sklearn** package. We will also write a custom function to build a classification matrix and use it for reporting the performance measures.

#### 3a. Confusion Matrix using sklearn

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
print("The model without dummy variable coding output: ")
confusion_matrix(lg_test_df.status, predict_test_df.predicted)
lg_reg_report = (classification_report(lg_test_df.status, predict_test_df.predicted))
print(lg_reg_report)


#### 3b Confusion Matrix using generic function

In [ ]:
def draw_cm( actual, predicted ):
    plt.figure(figsize=(9,9))
    cm = metrics.confusion_matrix( actual, predicted )
    sn.heatmap(cm, annot=True,  fmt='.0f', xticklabels = ["Joined", "Not Joined"] , 
               yticklabels = ["Joined", "Not Joined"],cmap = 'Blues_r')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Classification Matrix Plot', size = 15);
    plt.show()

The classification matrix plot as reported without dummy variable coding is:

In [ ]:
draw_cm( predict_test_df.actual, predict_test_df.predicted )

### 4. Performance Measure on the test set


In [ ]:
def measure_performance (clasf_matrix):
    measure = pd.DataFrame({
                        'sensitivity': [round(clasf_matrix[0,0]/(clasf_matrix[0,0]+clasf_matrix[0,1]),2)], 
                        'specificity': [round(clasf_matrix[1,1]/(clasf_matrix[1,0]+clasf_matrix[1,1]),2)],
                        'recall': [round(clasf_matrix[0,0]/(clasf_matrix[0,0]+clasf_matrix[0,1]),2)],
                        'precision': [round(clasf_matrix[0,0]/(clasf_matrix[0,0]+clasf_matrix[1,0]),2)],
                        'overall_acc': [round((clasf_matrix[0,0]+clasf_matrix[1,1])/
                                              (clasf_matrix[0,0]+clasf_matrix[0,1]+clasf_matrix[1,0]+clasf_matrix[1,1]),2)]
                       })
    return measure

In [ ]:
cm = metrics.confusion_matrix(predict_test_df.actual, predict_test_df.predicted)

lg_reg_metrics_df = pd.DataFrame(measure_performance(cm))
lg_reg_metrics_df

print( 'Total Accuracy sklearn: ',np.round( metrics.accuracy_score( lg_test_df.status, predict_test_df.predicted ), 2 ))




### 5. The optimal cut-off

We are going to use model without dummy variable coding to select the optimal cut-off. 


#### Select the optimal cut-off value, if:

> 1. Cost of Mis-classifying Not Joined as Joined is twice as costly as cost of micalssifying Joined as Not Joined
2. Both sensitivity and specificity are equally important

The best cut-off is the one which minimizes the misclassification cost (in case of **_option 1_**) or which maximizes the Youden's Index (in case of **_Option 2_**).

In [ ]:
lg_pred_prob_df = lg_reg_model.predict((lg_train_df))
n = len(lg_train_df)

costs =  pd.DataFrame.from_dict({'Joined': [0,1], 'Not Joined': [2,0]},
                    orient='index', columns=['Joined', 'Not Joined'])

print(costs)
costs.iloc[0][1] #to refer to specific value at a given position


Defining loop function to loop through float values


In [ ]:
def frange(start, stop, step):
     s = start
     while s < stop:
         yield s
         s += step

In [ ]:
#creating empty vectors to store the results.
cutoff = []
P11 = [] #correct classification of positive as positive
P00 = [] #correct classification of negative as negative
P10 = [] #misclassification of positive class to negative class
P01 = [] #misclassification of negative class to positive class


for i in frange(0.00, 1, 0.05):
    predicted_y = lg_pred_prob_df.map(lambda x: 'Joined' if x > i else 'Not Joined')
    tbl = metrics.confusion_matrix(lg_train_df.status, predicted_y)
    if ( i <= 1):
        j = int(20*i)
        P01.append(tbl[1,0]/(tbl[1,0] + tbl[1,1]))
        P00.append(tbl[1,1]/(tbl[1,0] + tbl[1,1]))
        P10.append(tbl[0,1]/(tbl[0,0] + tbl[0,1]))
        P11.append(tbl[0,0]/(tbl[0,0] + tbl[0,1]))
        cutoff.append(i)

d = {'cutoff':cutoff,'P10':P10,'P01': P01,'P00': P00,'P11':P11}
df_cost_table = pd.DataFrame(d, columns=['cutoff','P00','P01','P10','P11'])

In [ ]:
df_cost_table


The table summarizing the optimal cut-off value:

_write the cost.table into a csv file_


In [ ]:
df_cost_table['msclaf_cost'] = df_cost_table.P10*costs.iloc[0][1]+df_cost_table.P01*costs.iloc[1][0]
df_cost_table['youden_index'] = df_cost_table.P00+df_cost_table.P11 -1
df_cost_table

#write to csv
#df_cost_table.to_csv("optimal_Cutoff_caret.csv", sep=',')
#os.getcwd()


### 5. Confusion Matrix using Optimal Cut-off

The probability value along with the optimal cut-off can be used to build confusion matrix. We will use the **draw_cm** and **performance_measure** functions defined previously to report the performance of the model.

In [ ]:
predict_test_df['predicted'] = predict_test_df.predicted_prob.map(lambda x: 'Joined' if x > 0.8 else 'Not Joined') 
predict_test_df[0:10]

In [ ]:
draw_cm( predict_test_df.actual, predict_test_df.predicted )

In [ ]:
cm = metrics.confusion_matrix(predict_test_df.actual, predict_test_df.predicted)

pd.DataFrame(measure_performance(cm))



### 6. ROC Plot on the test data

ROCR package can be used to evaluate the model performace on the test data. The same package can also be used to get the model performace on the test data.



In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs, drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(10, 10))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return fpr, tpr, thresholds

Converting the labels to 0/1 code so that it can be passed to the sklearn identifies the positive and negative class.

In [ ]:
predict_test_df['predicted_code'] = predict_test_df.predicted.replace(('Joined', 'Not Joined'), (1, 0))
predict_test_df['actual_code'] = predict_test_df.actual.replace(('Joined', 'Not Joined'), (1, 0))


In [ ]:
fpr, tpr, thresholds = draw_roc(predict_test_df.actual_code, predict_test_df.predicted_prob )

In [ ]:
auc_score = metrics.roc_auc_score( predict_test_df.actual_code, predict_test_df.predicted_prob  )
round( float( auc_score ), 2 )


#### End of Document

***
***
